In [12]:
import os
import pickle
import random
from tqdm import tqdm
from joblib import Parallel, delayed

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from PIL import Image
from torchvision.transforms import Resize
import torch.nn.functional as F


import re

In [2]:
data_dir = "S:/Capston/data2"


SAVE_FOLDER = "S:/_tmp/"

In [3]:
series_meta_df = pd.read_csv(os.path.join(data_dir,'train_series_meta.csv'))

In [4]:
series_meta_df.shape

(4711, 4)

In [ ]:
# import csv
# # Write the list of .png files to a CSV file
# with open(os.path.join(data_dir,"images_file_name.csv"), 'w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerows(list_img)

In [36]:
list_img =os.listdir(os.path.join(data_dir,'train_images/'))
series_meta_df = pd.read_csv(os.path.join(data_dir,'train_series_meta.csv'))                             
lenght = series_meta_df.shape[0]     
img_paths = []
for index, row in series_meta_df.iterrows():
    scans = []
    patient = row["patient_id"]
    series = row["series_id"]
    pattern = "^" + str(int(patient)) + "_"+ str(int(series)) + "_"
    series_img_list  = [file for file in list_img if bool(re.match(pattern, file))]
    list_img = [file for file in list_img if file not in series_img_list ]
    series_img_paths = [os.path.join(data_dir, 'train_images/') + file for file in series_img_list]
    print(f"{index+1} / {lenght}, images in series:{len(series_img_paths)}", end='\r')
    img_paths.append(series_img_paths)

In [44]:
# import csv
# # Write the list of .png files to a CSV file
# with open(os.path.join(data_dir,"img_paths.csv"), 'w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerows(img_paths)

In [5]:
# import csv

# # Define the path to the CSV file
# csv_file_path = os.path.join(data_dir, "img_paths.csv")

# # Initialize a list to store the data from the CSV file
# img_paths = []

# # Read the CSV file
# with open(csv_file_path, 'r') as file:
#     reader = csv.reader(file)
#     for row in reader:
#         img_paths.append(row)

In [7]:
len(img_paths)

4711

In [15]:
def Series_to_tensor(paths):
    series_tensor = torch.zeros(store_res, store_res, len(paths))
    for index, path in enumerate(paths):
        image = Image.open(path)
        image = np.array(image)
        image = torch.tensor(image, dtype=torch.float32)

        #resize the 2d image 
        if image.shape != torch.Size([store_res, store_res]):
            image = image.unsqueeze(0).unsqueeze(0)
            image = F.interpolate(image, (store_res, store_res))
            image = image.squeeze(0).squeeze(0)
        series_tensor[:,:,index] = image

    # Resize the 3d image along the vertical dimension.            
    series_tensor = series_tensor.unsqueeze(0).unsqueeze(0)
    series_tensor = F.interpolate(series_tensor, (store_res, store_res, store_res))
    series_tensor = series_tensor.squeeze(0).squeeze(0)
    return series_tensor

In [19]:
# Defining resolution
store_res = 180
Root = "S:/Capston/data3" 
# Iterate over rows with tqdm progress bar
for index, row in tqdm(series_meta_df.iterrows(), total=len(series_meta_df)):
    
    patient = row["patient_id"]
    series = row["series_id"]

    series_tensor = Series_to_tensor(img_paths[index])
    torch.save(series_tensor, os.path.join(Root, str(int(patient)) + '_' + str(int(series)) + '.pt'))

100%|████████████████████████████████████████████████████████████████████████████| 4711/4711 [7:55:24<00:00,  6.05s/it]
